In [37]:
import pandas as pd
import string
from data_aquisition import check_download_extract
from data_preparation import DataExtractor
from data_matching import create_year_blocks
import nltk
import pandas as pd
import string

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
year_ranges = [(1995, 1996), (1997, 1998), (1999, 2000), (2001,2002), (2003,2004)]

df_dblp_path = pd.read_csv('data/DBLP_1995_2004.csv')
df_acm_path = pd.read_csv('data/ACM_1995_2004.csv')

#df_dblp_path = pd.read_csv('data/DBLP2.csv', engine='python')
#df_acm_path = pd.read_csv('data/ACM.csv', engine='python')

# get blocks as dictionary
result_blocks_dblp = create_year_blocks(df_dblp_path, year_ranges)
result_blocks_acm = create_year_blocks(df_acm_path, year_ranges)

print(result_blocks_dblp)
print(result_blocks_acm)
    
print("BLOCKS WERE CREATED AND SIMILARITY WILL BE CALCULATED")

[('Block_1',                            id  \
739  5390b52620f70186a0f01fe3   
752  5390b5df20f70186a0f09da7   
753  5390b5df20f70186a0f09db0   
754  5390b5df20f70186a0f09db2   
755  5390b5df20f70186a0f09db8   
..                        ...   
889  5390b72e20f70186a0f222f6   
890  5390b7fe20f70186a0f260af   
891  5390b7fe20f70186a0f260cb   
915  5390b9d520f70186a0f30cd2   
886  539087d920f70186a0d604f5   

                                                 title  \
739  Magic factoring of closure programs (extended ...   
752                    OODB indexing by class-division   
753  Recovery protocols for shared memory database ...   
754      The handwritten trie: indexing electronic ink   
755        The merge/purge problem for large databases   
..                                                 ...   
889                                  ACM SIGMOD Record   
890  Accessing information from globally distribute...   
891  Relational expressive power of constraint quer...   
915  Geogr

In [65]:
import nltk
import string

def tokenize(text):
    if isinstance(text, str):
        # Remove brackets and points
        text = text.translate(str.maketrans("", "", string.punctuation))
        
        # Tokenize and convert to uppercase
        return set(nltk.word_tokenize(text.upper()))
    else:
        return set()

In [79]:
def jaccard_similarity(list1, list2):
    s1 = set(map(str.upper, list1))
    s2 = set(map(str.upper, list2))
    
    intersection_size = len(s1.intersection(s2))
    union_size = len(s1.union(s2))
    
    # Check if the union is empty to avoid division by zero
    if union_size != 0:
        return float(intersection_size / union_size)
    else:
        # Handle the case where both sets are empty
        return 0.0

In [43]:
df_dblp_path = pd.read_csv('data/DBLP_1995_2004.csv')
df_acm_path = pd.read_csv('data/ACM_1995_2004.csv')

def find_matches(df1, df2):
    for idx1, row1 in df1.iterrows():
        for idx2, row2 in df2.iterrows():
            
            if row1['title'] == row2['title']:
                if row1['year'] == row2['year']:
                    sim_auth = jaccard_similarity(tokenize(row1['authors']), tokenize(row2['authors']))
                    if sim_auth >= 0.1:
                        sim_venue = jaccard_similarity(tokenize(row1['venue']), tokenize(row2['venue']))
                        if sim_venue >= 0.1:
                            print("This is a match!")
                            print(row1["id"])
                            print(row2["id"])


KeyboardInterrupt: 

In [41]:
result_blocks_acm[0]

('Block_1',
                             id  \
 486   53e9a301b7602d9702c098fe   
 430   53e99d6bb7602d97026235a8   
 423   53e99c6eb7602d9702521d3c   
 2002  573696106e3b12023e523505   
 2004  53e9b2eab7602d9703da7726   
 ...                        ...   
 162   53e9ab89b7602d9703535b9b   
 136   53e9b2e0b7602d9703d9658a   
 2061  53e9ac28b7602d97035e8af6   
 2092  53e99833b7602d9702058c0e   
 1471  53e9a060b7602d9702946a7c   
 
                                                   title  \
 486   OS Support for VLDBs: Unix Enhancements for th...   
 430        Managing Video Data in a Mobile Environment.   
 423   Data Management Research at The MITRE Corporat...   
 2002  Upsizing from File Server to Clent Server Arch...   
 2004   Enterprise Transaction Processing on Windows NT.   
 ...                                                 ...   
 162    To Table or Not to Table: a Hypertabular Answer.   
 136   OLAP, Relational, and Multidimensional Databas...   
 2061    Query Processing 

In [58]:
result_blocks_acm[0].loc[0, 1:]

AttributeError: 'tuple' object has no attribute 'loc'

In [38]:
def trigram_similarity(str1, str2):
    def get_trigrams(text):
        trigrams = set()
        text = "  " + text + "  "
        for i in range(len(text) - 2):
            trigram = text[i:i+3]
            trigrams.add(trigram)
        return trigrams
    
    trigrams1 = get_trigrams(str1)
    trigrams2 = get_trigrams(str2)

    print(f"Trigrams for {str1}: {trigrams1}")
    print(f"Trigrams for {str2}: {trigrams2}")
    print(len(trigrams1))
    print(len(trigrams2))

    intersection = len(trigrams1.intersection(trigrams2))
    union = len(trigrams1) + len(trigrams2)

    if union == 0:
        return 0.0
    else:
        return 2 * intersection / union

# Example usage
string1 = "hello world"
string2 = "hola mundo"
similarity_score = trigram_similarity(string1, string2)
print(f"\nTrigram Similarity: {similarity_score}")


Trigrams for hello world: {'llo', 'rld', '  h', ' he', ' wo', 'wor', 'o w', 'lo ', 'orl', 'ld ', 'd  ', 'ell', 'hel'}
Trigrams for hola mundo: {' mu', '  h', 'hol', 'o  ', 'la ', 'mun', 'do ', ' ho', 'ola', 'ndo', 'a m', 'und'}
13
12

Trigram Similarity: 0.08


In [39]:
sim = trigram_similarity('Matt Böhm', 'Matt Boehm')
print(sim)

Trigrams for Matt Böhm: {'hm ', ' Ma', 'tt ', 'm  ', ' Bö', 'Mat', 'Böh', 't B', 'att', '  M', 'öhm'}
Trigrams for Matt Boehm: {'Boe', ' Ma', 'hm ', 'tt ', 'm  ', 'oeh', 'Mat', 'ehm', 't B', 'att', '  M', ' Bo'}
11
12
0.6956521739130435


In [40]:
def tokenize(text):
    if isinstance(text, str):
        # Remove brackets and points
        text = text.translate(str.maketrans("", "", string.punctuation))
        
        # Tokenize and convert to uppercase
        return set(nltk.word_tokenize(text.upper()))
    else:
        return set()

In [41]:
def tokenize(text):
    if isinstance(text, str):
        # Remove brackets and points
        text = text.translate(str.maketrans("", "", string.punctuation))
        
        # Tokenize and convert to uppercase
        return set(nltk.word_tokenize(text.upper()))
    else:
        return set()
    
def jaccard_similarity(list1, list2):
    s1 = tokenize(list1)
    s2 = tokenize(list2)
    
    intersection_size = len(s1.intersection(s2))
    union_size = len(s1.union(s2))
    
    # Check if the union is empty to avoid division by zero
    if union_size != 0:
        return float(intersection_size / union_size)
    else:
        # Handle the case where both sets are empty
        return 0.0

In [42]:
sim = jaccard_similarity("Matt Böhm", "Matt Boehm")
print(sim)

0.3333333333333333
